# PCA with mutate normalize

In [ ]:
import pymysql.cursors
import pandas as pd
import pandas_profiling
import numpy as np
from numpy import isnan
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE, SelectKBest, f_regression
# from sklearn.tree import DecisionTreeClassifier #타겟 변수가 0과 1일 때
from sklearn.svm import SVR #타겟 변수가 numeric 일 때
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets import make_classification, make_regression
from sklearn.decomposition import PCA

## 1. Data read from sql

In [ ]:
conn = pymysql.connect(host='localhost', user='amps',
                       password='123123', db='tip', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)
try:
   with conn.cursor() as curs:
      sql = "SELECT * FROM tips;"
      curs.execute(sql)
      rs = curs.fetchall()

      # DB에서 받아온 값을 DataFrame에 넣음
      df = pd.DataFrame(rs)
      # print(df)
finally:
   conn.close()

In [ ]:
df.columns

## 2. encoding and add variable

In [ ]:
#범주형 변수 연속형 변수로 인코딩
df['sex'].replace({'Female':0, 'Male':1}, inplace = True)
df['smoker'].replace({'No':0, 'Yes':1}, inplace = True)
df['day'].replace({'Thur':0, 'Fri':1, 'Sat':2, 'Sun':3}, inplace = True)
df['time'].replace({'Lunch':0, 'Dinner':1}, inplace = True)
# df.replace('',np.nan, inplace=True) #''데이터가 있으면 nan값으로 대체 (공백 데이터가 있으면 아래 impute과정에서 오류 발생)
df['tip_rate'] = ((df['tip']/df['total_bill'])*100).round(2)
print(df.isnull().sum()) # df에 null값이 있는지 확인

## 3. nan processing

In [ ]:
data = df.values # DataFrame to array
imputer = SimpleImputer(strategy='median') #중앙값으로 null값 대체
imputer.fit(data)
data_trans = imputer.transform(data) #null값이 없는 imputed dataset
# data_trans[:,5] #데이터에 null값이 있는지 확인
print('flatten 후 null값 확인 : ',sum(isnan(data_trans).flatten()))

## 3_1 before normalize

In [26]:
#define the transform
trans = PCA(n_components = 4) #feature를 4개만 뽑음
#transform the data
X_dim = trans.fit_transform(data_trans)
#summarize data after the transform
print(X_dim[:3,:]) #4개의 feature의 첫 3줄이 나옴

[[ -0.66990102 -10.63318832  -1.43736174  -0.29725425]
 [  8.88262165  -3.23598195  -1.66428915   0.96927803]
 [ -1.02247662   1.09963706  -1.29568038   0.37640344]]


In [ ]:
pca_trans = pd.DataFrame(X_dim)

In [ ]:
pca_trans.profile_report()

## 4. normalize

In [20]:
norm = MinMaxScaler()

In [21]:
X_norm = norm.fit_transform(data_trans)

In [22]:
df_norm = pd.DataFrame(X_norm, columns = ['total_bill','tip', 'sex', 'smoker', 'day', 'time', 'size', 'tip_rate'])

In [23]:
df_norm.describe()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
count,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000
mean,0.350201,0.221772,0.646586,0.373494,0.574297,0.726908,0.317269,0.184990
std,0.184639,0.152905,0.478993,0.484706,0.384442,0.446445,0.192770,0.090103
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.216171,0.111111,0.000000,0.000000,0.000000,0.000000,0.200000,0.139025
50%,0.308965,0.222222,1.000000,0.000000,0.666667,1.000000,0.200000,0.177857
75%,0.439673,0.278889,1.000000,1.000000,1.000000,1.000000,0.400000,0.229584
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## 5. before pca

## 6. PCA

In [27]:
#define the transform
trans = PCA(n_components = 4) #feature를 4개만 뽑음
#transform the data
X_norm_dim = trans.fit_transform(X_norm)
#summarize data after the transform
print(X_norm_dim[:3,:]) #4개의 feature의 첫 3줄이 나옴

[[-0.12119356 -0.32720341  0.81792869 -0.20079806]
 [-0.55938836 -0.40878586 -0.0751585  -0.21156993]
 [-0.58625278 -0.40302457 -0.08310396  0.02085156]]


In [ ]:
trans.components_

In [ ]:
trans.explained_variance_ratio_.cumsum()

In [ ]:
trans.explained_variance_ratio_

In [ ]:
trans.explained_variance_ratio_.sum()

In [ ]:
'''
import pandas as pd
import pylab as pl
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn import preprocessing

df1 = pd.DataFrame(data_trans, columns = ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'])
df1.head()

# PCA
trans = PCA(n_components=4)
trans.fit_transform(X_dim)

# Dump components relations with features:
# print(pd.DataFrame(trans.components_,columns=X_dim.columns))
'''

In [ ]:
pca_trans = pd.DataFrame(X_dim)

In [ ]:
pca_trans.profile_report()